In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import datetime as dt
import calendar,warnings,itertools,matplotlib,keras,shutil
import tensorflow as tf
import statsmodels.api as sm
from datetime import datetime
from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict
from sklearn import svm,metrics,tree,preprocessing,linear_model
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge,LinearRegression,LogisticRegression,ElasticNet, Lasso
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier, GradientBoostingRegressor,BaggingClassifier,ExtraTreesClassifier
from sklearn.metrics import accuracy_score,mean_squared_error,recall_score,confusion_matrix,f1_score,roc_curve, auc
from sklearn.datasets import load_iris,make_regression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.kernel_ridge import KernelRidge
from keras import Sequential
from keras.layers import Dense
from IPython.core import display as ICD

#Hiding the warnings
warnings.filterwarnings('ignore')

file_path = r"C:\Users\23851\PycharmProjects\Dataguan\Dissertation\DataCoSupplyChainDataset.csv"
dataset = pd.read_csv(file_path,encoding= 'unicode_escape')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
print(dataset.head(30))

dataset.shape


dataset.apply(lambda x: sum(x.isnull())) #Checking missing values
# Adding first name and last name together to create new column
dataset['Customer Full Name'] = dataset['Customer Fname'].astype(str)+dataset['Customer Lname'].astype(str)

data=dataset.drop(['Customer Email','Product Status','Customer Password','Customer Street','Customer Fname','Customer Lname',
           'Latitude','Longitude','Product Description','Product Image','Order Zipcode','shipping date (DateOrders)'],axis=1)
data['Customer Zipcode']=data['Customer Zipcode'].fillna(0)#Filling NaN columns with zero

# corr = data.corr()
# corr.style.background_gradient(cmap='coolwarm')
# plt.show()
#
#
#
# market = data.groupby('Market') #Grouping by market
# region = data.groupby('Order Region')
# plt.figure(1)
# market['Sales per customer'].sum().sort_values(ascending=False).plot.bar(figsize=(12,6), title="Total sales for all markets")
# plt.figure(2)
# region['Sales per customer'].sum().sort_values(ascending=False).plot.bar(figsize=(12,6), title="Total sales for all regions")
#
# #Grouping all categories
# # cat = data.groupby('Category Name')
# # plt.figure(1)
# # # Total sum of sales for all categories
# # cat['Sales per customer'].sum().sort_values(ascending=False).plot.bar(figsize=(12,6), title="Total sales")
# # # Mean sales for all categories
# # plt.figure(2)
# # cat['Sales per customer'].mean().sort_values(ascending=False).plot.bar(figsize=(12,6), title="Average sales")
# # plt.figure(3)
# # # Mean prices for all categories
# # cat['Product Price'] = pd.to_numeric(cat['Product Price'], downcast="float")
# # cat['Product Price'].mean().sort_values(ascending=False).plot.bar(figsize=(12,6), title="Average price")
#
#
#
# data.plot(x='Product Price', y='Sales per customer',linestyle='dotted',
#      markerfacecolor='blue', markersize=12)
# plt.title('Product Price vs Sales per customer')#title
# plt.xlabel('Product Price')  # X-axis title
# plt.ylabel('Sales per customer') # Y=axis title
# plt.show()



# data['order_week_day'] = pd.DatetimeIndex(data['order date (DateOrders)']).weekday_name
# print(data['order_week_day'])
#
# days=data.groupby("order_week_day")

train_data=data.copy()

train_data['fraud'] = np.where(train_data['Order Status'] == 'SUSPECTED_FRAUD', 1, 0)
train_data['late_delivery']=np.where(train_data['Delivery Status'] == 'Late delivery', 1, 0)

train_data.drop(['Delivery Status','Late_delivery_risk','Order Status','order date (DateOrders)'], axis=1, inplace=True)







# create the Labelencoder object
le = preprocessing.LabelEncoder()
#convert the categorical columns into numeric
train_data['Customer Country']  = le.fit_transform(train_data['Customer Country'])
train_data['Market']            = le.fit_transform(train_data['Market'])
train_data['Type']              = le.fit_transform(train_data['Type'])
train_data['Product Name']      = le.fit_transform(train_data['Product Name'])
train_data['Customer Segment']  = le.fit_transform(train_data['Customer Segment'])
train_data['Customer State']    = le.fit_transform(train_data['Customer State'])
train_data['Order Region']      = le.fit_transform(train_data['Order Region'])
train_data['Order City']        = le.fit_transform(train_data['Order City'])
train_data['Category Name']     = le.fit_transform(train_data['Category Name'])
train_data['Customer City']     = le.fit_transform(train_data['Customer City'])
train_data['Department Name']   = le.fit_transform(train_data['Department Name'])
train_data['Order State']       = le.fit_transform(train_data['Order State'])
train_data['Shipping Mode']     = le.fit_transform(train_data['Shipping Mode'])
# train_data['order_week_day']    = le.fit_transform(train_data['order_week_day'])
train_data['Order Country']     = le.fit_transform(train_data['Order Country'])
train_data['Customer Full Name']= le.fit_transform(train_data['Customer Full Name'])

#display the initial records
train_data.head()


# Classification Models
#All columns expect fraud
xf=train_data.loc[:, train_data.columns != 'fraud']
#Only fraud column
yf=train_data['fraud']
#Splitting the data into two parts in which 80% data will be used for training the model and 20% for testing
xf_train, xf_test,yf_train,yf_test = train_test_split(xf,yf,test_size = 0.2,random_state = 42)
#All columns expect fraud
xl=train_data.loc[:, train_data.columns != 'late_delivery']
#Only fraud column
yl=train_data['late_delivery']
#Splitting the data into two parts in which 80% data will be used for training the model and 20% for testing
xl_train, xl_test,yl_train,yl_test = train_test_split(xl,yl,test_size = 0.2, random_state = 42)

        Type  Days for shipping (real)  Days for shipment (scheduled)  Benefit per order  Sales per customer    Delivery Status  Late_delivery_risk  Category Id   Category Name   Customer City Customer Country Customer Email Customer Fname  Customer Id Customer Lname Customer Password Customer Segment Customer State              Customer Street  Customer Zipcode  Department Id Department Name   Latitude   Longitude        Market  Order City  Order Country Order Customer Id order date (DateOrders)  Order Id  Order Item Cardprod Id  Order Item Discount  Order Item Discount Rate  Order Item Id  Order Item Product Price  Order Item Profit Ratio  Order Item Quantity   Sales  Order Item Total Order Profit Per Order    Order Region             Order State     Order Status  Order Zipcode  Product Card Id Product Category Id Product Description                                 Product Image  Product Name Product Price Product Status shipping date (DateOrders)   Shipping Mode
0      DEBIT        